# 前面章節沒有實作用於取得股價的工具
# 透過 yf 取得股價資料的類別 StockDataManager

In [103]:
import yfinance as yf
import pandas as pd
import numpy as np

class StockDataManager:
    """
    負責使用 yfinance 下載、儲存和提供股票 K 線資料的類別。
    """
    def __init__(self, symbols, start_date=None, end_date=None):
        """
        初始化管理器，並下載指定股票的資料。
        
        參數:
            symbols (list): 要下載的股票代碼列表 (e.g., ["TSLA", "AAPL"])。
            start_date (str): 資料下載的起始日期。
            end_date (str, optional): 資料下載的結束日期。
        """
        self.all_symbols = symbols
        self.start_date = start_date
        self.end_date = end_date
        self.stock_data = {}
        
        self._download_data()

    def _download_data(self):
        """
        實際執行 yfinance 下載資料的內部方法。
        """
        print(f"正在使用 yfinance 下載 {len(self.all_symbols)} 檔股票資料...")
        
        for symbol in self.all_symbols:
            try:
                # progress=False 避免輸出過多進度條
                df = yf.download(symbol, 
                                 start=self.start_date, 
                                 end=self.end_date, 
                                 progress=False, 
                                 auto_adjust=True)
                
                if df.empty:
                    print(f"警告: {symbol} 未能下載到資料，跳過。")
                    continue
                
                # 儲存 K 線資料
                self.stock_data[symbol] = df
                
            except Exception as e:
                print(f"下載 {symbol} 時發生錯誤: {e}")

        # 打印下載摘要
        print("---" * 10)
        print(f"已成功下載 {len(self.stock_data)} 檔股票資料")
        for symbol, df in self.stock_data.items():
            print(f"  - {symbol}: {len(df)} 筆資料，日期範圍 {df.index[0].date()} 至 {df.index[-1].date()}")
        print("---" * 10)

    def get_kl_pd(self, symbol):
        """
        提供給選股因子 fit_pick 方法使用的 K 線 DataFrame。
        
        參數:
            symbol (str): 股票代碼。

        回傳:
            pd.DataFrame: 股票的 K 線資料，如果資料不存在，回傳空 DataFrame。
        """
        return self.stock_data.get(symbol, pd.DataFrame())
    
    
# =========================================================
print("="*60)
print("StockDataManager 使用範例")
print("="*60)
# =========================================================

# 使用範例    
# 初始化 StockDataManager
# 1. 定義要下載的股票代碼和時間範圍
symbols_to_download = ["TSLA", "AAPL", "MSFT", "BABA"]
start_date = "2023-01-01"
# 結束日期如果為 None，則預設下載到最新日期

print("## 步驟 1: 實例化 StockDataManager 並下載資料")
data_manager = StockDataManager(symbols_to_download, start_date=start_date)

# 2. 存取單一股票的 K 線資料
target_symbol = "AAPL"
aapl_df = data_manager.get_kl_pd(target_symbol)

print(f"\n## 步驟 2: 存取 {target_symbol} 的 K 線資料")
if not aapl_df.empty:
    print(f"{target_symbol} 資料前 5 行:")
    print(aapl_df.head())
    
    # 存取收盤價序列
    close_series = aapl_df['Close']
    print(f"\n{target_symbol} 收盤價序列長度: {len(close_series)}")
else:
    print(f"{target_symbol} 的資料不存在或下載失敗。")


# 3. 遍歷所有已下載的股票
print("\n## 步驟 3: 遍歷所有股票並檢查資料大小")
for symbol in symbols_to_download:
    df = data_manager.get_kl_pd(symbol)
    if not df.empty:
        print(f"  - {symbol} 的資料筆數: {len(df)}")
    else:
        print(f"  - {symbol} 的資料為空。")

StockDataManager 使用範例
## 步驟 1: 實例化 StockDataManager 並下載資料
正在使用 yfinance 下載 4 檔股票資料...
------------------------------
已成功下載 4 檔股票資料
  - TSLA: 723 筆資料，日期範圍 2023-01-03 至 2025-11-18
  - AAPL: 723 筆資料，日期範圍 2023-01-03 至 2025-11-18
  - MSFT: 723 筆資料，日期範圍 2023-01-03 至 2025-11-18
  - BABA: 723 筆資料，日期範圍 2023-01-03 至 2025-11-18
------------------------------

## 步驟 2: 存取 AAPL 的 K 線資料
AAPL 資料前 5 行:
Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2023-01-03  123.211205  128.954553  122.324579  128.343772  112117500
2023-01-04  124.482033  126.747853  123.221057  125.004155   89113600
2023-01-05  123.161957  125.871086  122.905826  125.240599   80962700
2023-01-06  127.693596  128.353637  123.033897  124.137254   87754700
2023-01-09  128.215698  131.427258  127.959568  128.530950   70790800

AAPL 收盤價序列長度: 723

## 步驟 3: 遍歷所有股票並檢查資料大小
  - TSL


# 輔助工具及裝飾器 (RegressionUtil 類別 & reversed_result 裝飾器方法)

In [104]:
import numpy as np
from functools import wraps
# *** 必須新增以下 import ***
import statsmodels.api as sm 
from statsmodels import regression # 為了使用 regression.linear_model.OLS

class RegressionUtil:
    """
    計算標準化後的股價趨勢角度（可跨股票比較）。
    """
    @staticmethod
    def calc_regress_deg(data, symbol=None, show=False): 
        """
        用線性回歸 + 標準化計算趨勢角度。
        """
        y_arr = data.values.astype(float)  
        n = len(y_arr)

        if n < 2:
            return 0.0

        # --- 1. X 與 Y 同時標準化（0~1 區間） ---
        x_raw = np.arange(n)

        x_norm = (x_raw - x_raw.min()) / (x_raw.max() - x_raw.min())
        y_norm = (y_arr - y_arr.min()) / (y_arr.max() - y_arr.min())

        # --- 2. OLS 回歸 ---
        X = sm.add_constant(x_norm)
        model = regression.linear_model.OLS(y_norm, X).fit()

        slope = model.params[1]

        # --- 3. 斜率轉角度：正確公式 angle = atan(slope) ---
        angle = np.degrees(np.arctan(slope))

        # --- 4. 畫圖（選擇性） ---
        if show:
            import matplotlib.pyplot as plt
            
            intercept = model.params[0]
            reg_y_fit = slope * x_norm + intercept

            plt.figure(figsize=(10, 5))
            plt.plot(x_norm, y_norm, label='Normalized Price', linewidth=2)
            plt.plot(x_norm, reg_y_fit, label='Regression Line', linestyle='--')

            symbol_str = f"{symbol} - " if symbol else ""
            plt.title(f"{symbol_str}Trend Angle (deg) = {angle:.2f}")
            plt.xlabel("Normalized Time")
            plt.ylabel("Normalized Price")
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.show()

        return float(angle)

    
def reversed_result(func):
    """如果 self.reversed 為 True，則翻轉 fit_pick 的布林返回值"""
    @wraps(func)
    def wrapper(self, kl_pd, target_symbol):
        result = func(self, kl_pd, target_symbol)
        if hasattr(self, 'reversed') and self.reversed:
            return not result
        return result
    return wrapper

# 抽象類別 StockPickerBase

In [105]:
from abc import ABC, abstractmethod

class StockPickerBase(ABC):
    """
    選股因子的抽象基類 (Abstract Base Class)。
    強制所有子類必須實作 fit_pick 方法。
    """
    def __init__(self, capital, benchmark, **kwargs):
        # 假設這些是回測系統中的核心物件
        self.capital = capital
        self.benchmark = benchmark
        self._init_self(**kwargs)

    # 普通方法，子類可選擇性覆蓋
    def _init_self(self, **kwargs):
        """子類初始化參數"""
        pass

    @abstractmethod
    def fit_pick(self, kl_pd, target_symbol):
        """
        核心選股邏輯：判斷是否應選中該股票。子類必須實作。
        
        參數:
            kl_pd (pd.DataFrame): 股票的 K 線資料。
            target_symbol (str): 股票代碼。
        回傳:
            bool: 如果滿足選股條件返回 True，否則返回 False。
        """
        pass

# 具體的選股因子類 (RegressAnglePicker)

In [106]:
import numpy as np

class RegressAnglePicker(StockPickerBase):
    """
    基於股票 K 線收盤價的線性迴歸趨勢角度進行篩選的選股因子。
    """
    def _init_self(self, **kwargs):
        """初始化角度閥值、reversed 和 show 屬性"""
        
        self.threshold_ang_min = kwargs.get('threshold_ang_min', -np.inf)
        self.threshold_ang_max = kwargs.get('threshold_ang_max', np.inf)
        
        self.reversed = kwargs.get('reversed', False)
        
        # *** 新增：接收並儲存 show 參數 ***
        # 預設為 False，除非使用者明確設定為 True
        self.show_plot = kwargs.get('show', False) 

    @reversed_result
    def fit_pick(self, kl_pd, target_symbol):
        if kl_pd.empty:
            return False
            
        # ... (close_data 獲取邏輯不變)
        try:
            close_data = kl_pd['Close']
        except KeyError:
            try:
                close_data = kl_pd['close']
            except KeyError:
                return False 
            
        ang = RegressionUtil.calc_regress_deg(
            close_data, 
            symbol=target_symbol,
            show=self.show_plot
        )
        
        # 根據參數進行角度條件判斷
        return self.threshold_ang_min < ang < self.threshold_ang_max

# 選股執行器 StockPickerWorker

In [107]:
import copy
import pandas as pd # 需要引入 pandas

class StockPickerWorker:
    """模擬回測系統中的選股核心，負責遍歷股票並應用選股因子。"""
    def __init__(self, data_manager, stock_pickers):
        self.data_manager = data_manager
        self.stock_pickers = stock_pickers
        self.picker_instances = self._init_pickers()
        self.choice_symbols = []
        # 新增屬性：用於儲存詳細的因子篩選結果 DataFrame
        self.factor_results_df = None 
        # 新增屬性：用於儲存所有因子的名稱 (用於 DataFrame 欄位)
        self.factor_names = [f"{p['class'].__name__}_{i}" for i, p in enumerate(stock_pickers)]


    def _init_pickers(self):
        """根據配置實例化選股因子"""
        picker_list = []
        for config in self.stock_pickers:
            picker_config = copy.deepcopy(config)
            picker_class = picker_config['class']
            del picker_config['class']
            picker = picker_class(None, None, **picker_config)
            picker_list.append(picker)
        return picker_list

    def fit(self):
        """執行選股過程，並將結果儲存到 self.factor_results_df"""
        print("\n--- 正在執行選股過程並記錄詳細因子結果 ---")
        
        results_list = []
        
        for symbol in self.data_manager.stock_data.keys():
            kl_pd = self.data_manager.get_kl_pd(symbol)
            is_picked_overall = True
            
            # 初始化該股票的結果字典
            result_row = {'symbol': symbol}
            
            # --- 1. 遍歷並紀錄每個因子的結果 ---
            for i, picker in enumerate(self.picker_instances):
                factor_passed = picker.fit_pick(kl_pd, symbol)
                factor_name = self.factor_names[i]
                
                result_row[factor_name] = factor_passed
                
                if not factor_passed:
                    is_picked_overall = False
            
            # --- 2. 紀錄關鍵指標和最終篩選結果 ---
            # current_angle = RegressionUtil.calc_regress_deg(kl_pd['Close'])
            
            # result_row['趨勢角度'] = current_angle
            result_row['最終選中'] = is_picked_overall
            
            results_list.append(result_row)
            
            # print(f"  > {symbol}: 趨勢角度={current_angle:.2f}°, 最終結果={is_picked_overall}")
            
            if is_picked_overall:
                self.choice_symbols.append(symbol)

        # --- 3. 轉換為 DataFrame 並儲存 ---
        self.factor_results_df = pd.DataFrame(results_list)
        # 將 symbol 設為索引，更方便檢視
        self.factor_results_df.set_index('symbol', inplace=True)
        
        print("\n--- 詳細因子結果已儲存至 self.factor_results_df ---")

In [112]:
# 1. 定義測試股票和時間範圍
TEST_SYMBOLS = ["TSLA", "AAPL", "MSFT", "GOOG", "AMD"]
START_DATE = "2024-01-01"

# 2. 實例化資料管理器，下載真實股價資料
print("## 步驟 1: 下載資料")
data_manager = StockDataManager(TEST_SYMBOLS, start_date=START_DATE)

# 3. 定義選股條件: 篩選出 **溫和上升趨勢** 的股票
select_config = [{
                    'class': RegressAnglePicker, 
                    'threshold_ang_min': 20.0,  
                    'reversed': False
                },
                # 多給一個因子測試看看多因子選股
                # {
                #      'class': RegressAnglePicker, 
                #      'threshold_ang_min': 10.0, 
                #      'threshold_ang_max': 15.0, 
                #      'reversed': False,
                #      'show': True
                #  }
            ]

# 4. 實例化選股執行器
picker_worker = StockPickerWorker(data_manager, select_config)

# 5. 執行選股
picker_worker.fit()

# 6. 打印選股結果
print("\n--- 選股結果 ---")
print(picker_worker.choice_symbols)

# 5. 打印最終結果
print("\n--- 最終選股結果 ---")
print(picker_worker.choice_symbols)

# 6. 看一下選股結果的 DataFrame
picker_worker.factor_results_df

## 步驟 1: 下載資料
正在使用 yfinance 下載 5 檔股票資料...
------------------------------
已成功下載 5 檔股票資料
  - TSLA: 473 筆資料，日期範圍 2024-01-02 至 2025-11-18
  - AAPL: 473 筆資料，日期範圍 2024-01-02 至 2025-11-18
  - MSFT: 473 筆資料，日期範圍 2024-01-02 至 2025-11-18
  - GOOG: 473 筆資料，日期範圍 2024-01-02 至 2025-11-18
  - AMD: 473 筆資料，日期範圍 2024-01-02 至 2025-11-18
------------------------------

--- 正在執行選股過程並記錄詳細因子結果 ---

--- 詳細因子結果已儲存至 self.factor_results_df ---

--- 選股結果 ---
['TSLA', 'AAPL', 'MSFT', 'GOOG']

--- 最終選股結果 ---
['TSLA', 'AAPL', 'MSFT', 'GOOG']


,RegressAnglePicker_0,最終選中
symbol,,
TSLA,True,True
AAPL,True,True
MSFT,True,True
GOOG,True,True
AMD,False,False
